In [1]:
from Crypto.Util.number import sieve_base, GCD, isPrime, long_to_bytes, bytes_to_long, inverse, getPrime
import random

In [2]:
def miller_rabin_test_fixed(n, a_list):
    '''returns Composite or Probably prime'''
    
    #check parity
    if not n & int(1):
        return 'Composite'
    
    r = 0
    s = n-1
    #write n-1 = 2^r*s
    while not s & int(1):
        s = s>>1
        r+=1
    assert(pow(2, r)* s == n-1)
    
    for a in a_list:
        x = pow(a, s, n)
        if x == 1 or x == n-1:
            continue #search for another witness
        for _ in range(r):
            x = pow(x, 2, n)
            if x == n-1:
                break 
        else:
             #if it doesnt break, neither condition is satisfied =>  this executes => we found a composite
            return "Composite"
    return "Probably prime"
    

## Fixed bases -> Bleichenbacher

https://www.semanticscholar.org/paper/Breaking-a-Cryptographic-Protocol-with-Pseudoprimes-Bleichenbacher/e9f1f083adc1786466d344db5b3d85f4c268b429?p2df

**(Korselt's Criterion)**: A composite integer $n$ is a Carmichael number if and only if the following two conditions are satisfied:
- $n$ is square-free, i.e., $n$ is not divisible by the square of any prime; 
- $p−1$ divides $n−1$ for every prime divisor $p$ of $n$

Erdos's idea to construct pseudoprimes
- Choose $M \in \mathbb{Z}$ that has many divisors
- Let $R$ be the set of primes $r$ such that $r−1$ is a divisor of $M$. 
- If a subset $T⊂R$ can be found such that
$$ C = \underset {r\in T}{\prod} r \equiv 1 \bmod M \ \ \ (1)$$
Then C is a Carmichael number because C satisfies the Korselt's criterion 

**Note**: One can hope to find such sets $$ if $R$ contains more than about $\log_2(M)$ primes


Additionally, a Carmichael number $C$ is a strong pseudoprime for a base $a$ if the order of $a$ modulo $r$ is divisible by the same power of 2 for all primes factors $r$ of $C$.  
If all prime factors $r$ are congruent 3 modulo 4 then this condition is satisfied when $a$ is a quadratic residue modulo either all prime factors $r$ or none at all, because in that case the order of $a$ modulor is either even or odd for all $r$

**Algorithm**
- Choose $M \in \mathbb{Z}$
- Find a set $R$ of primes s.t $\forall $a$ \in $A$ \ \exists c_i \in \{-1,1\} \text{ with } \left(\dfrac {a_i} r\right) = c_i \ \forall r \in R$
- Find a subset $T \subset R$ that can satisfy equation (1)

In [3]:
def generate_M(a_list, prime_idx_bound, random_bound):
    M = 1
    for a in a_list:
        M *= pow(a, random.randint(1, 3))
    p = a_list[-1]
    for i in range(random_bound):
        p = next_prime(p)
        t = random.choice([0, 1])
        if t == 0:
            i-=1
            continue
        
        M*= p
        
    return M

In [4]:
def find_set_R(a_list, M, r_low, r_high):
    r_list = []
    for r in divisors(M):
        r+=1
        if r < r_low:
            continue
        if r > r_high:
            break
        if isPrime(r):
            for a in a_list:
                if legendre_symbol(a, r) not in [-1, 1]:
                    break
            else: #if it (bi / r) is -1 or 1 for all bases 
                r_list.append(r)
    return r_list

In [33]:
def find_subsets_T(R, M):
    #we use a meet in the middle approach
    R1 = R[:len(R)//4]
    R2 = R[len(R)//4:]
    
    #precompute table for the first subset
    print("computing table - searching through %s subsets" % len(Subsets(R1)))
    #table = {}
    table  = []
    table2 = []
    for T1 in Subsets(R1):
        if len(T1) > 1:
            #print(T1)
            #products are unique since they are made from primes
            #table[inverse_mod(product(T1), M)] = T1
            t = product(T1)
            table.append(inverse_mod(t, M))
            table2.append(t)
    
    print("table computed, length %s" % len(table))
    print("Starting search through the table")
    for T2 in Subsets(R2):
        if len(T2) > 1:
            t = product(T2) % M
#             T1 = table.get(pr, 0)
#             if T1 != 0:
#                 T = list(set(T1 + T2))
#                 C = product(T)
#                 return C
            if t in table:
                idx = table.index(t)
                C = table2[idx] * t
                return C
    return -1
            
    
            
    

In [20]:
a_list = [2,3,5]
a_list = [int(a) for a in a_list]

In [44]:
M = generate_M(a_list, 20, 2)
M

346500

In [45]:
#M = 2*5^3*7^2*11^2*13*17*19*23*29*31*37*41*61

In [46]:
factor(M)

2^2 * 3^2 * 5^3 * 7 * 11

In [47]:
r_list = find_set_R(a_list, M, 256, 2^60)

In [48]:
C= find_subsets_T(r_list, M)

computing table - searching through 64 subsets
table computed, length 57
Starting search through the table


In [49]:
factor(C)

331 * 397 * 421 * 463 * 631 * 661 * 14851

In [50]:
miller_rabin_test_fixed(C, a_list)

'Composite'

In [52]:
from sage.crypto.util import carmichael_lambda

In [53]:
carmichael_lambda(C)

207900